In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

In [115]:
ADMISSIONS = pd.read_csv('ADMISSIONS.csv')
PATIENTS = pd.read_csv('PATIENTS.csv')
PRESCRIPTIONS = pd.read_csv('PRESCRIPTIONS.csv')
PROCEDURES_ICD = pd.read_csv('PROCEDURES_ICD.csv')
SERVICES = pd.read_csv('SERVICES.csv')
TRANSFERS = pd.read_csv('TRANSFERS.csv')
CAREGIVERS = pd.read_csv('CAREGIVERS.csv')

In [54]:
sql = """Select c.subject_id,c.hadm_id, c.hospital_expire_flag, strftime('%Y',c.admittime) - strftime('%Y',c.dob) as age
from 
(Select a.subject_id, a.hadm_id, a.hospital_expire_flag, b.dob, a.admittime
from ADMISSIONS a
left join PATIENTS b
on a.subject_id = b.subject_id) c"""
df = pysqldf(sql)
df

,subject_id,hadm_id,hospital_expire_flag,age
0,10006,142345,0,70
1,10011,105331,1,36
2,10013,165520,1,87
3,10017,199207,0,74
4,10019,177759,1,49
...,...,...,...,...
124,44083,198330,0,55
125,44154,174245,1,300
126,44212,163189,0,45
127,44222,192189,0,73


In [112]:
sql1 = """select d.subject_id, d.hadm_id, d.num_transfers,d.curr_careunit
from
(select *, ROW_NUMBER() over (partition by c.hadm_id order by c.outtime desc) as rownum 
from
(select *
from
(select a.subject_id, a.hadm_id, sum(transfer) over (partition by a.hadm_id) num_transfers, a.curr_careunit, los, outtime
from
(Select subject_id, hadm_id, case when eventtype = 'transfer' then 1 else 0 end as transfer, curr_careunit, los, outtime
from TRANSFERS) a) b
where b.curr_careunit is not null) c) d
where d.rownum = 1"""
df1 = pysqldf(sql1)
df1

,subject_id,hadm_id,num_transfers,curr_careunit
0,10056,100375,1,MICU
1,42430,100969,1,SICU
2,41914,101361,2,MICU
3,42135,102203,6,MICU
4,44228,103379,2,SICU
...,...,...,...,...
124,44083,198330,3,CCU
125,40177,198480,4,MICU
126,10076,198503,3,MICU
127,10017,199207,4,CCU


In [113]:
sql2 = """select a.subject_id, a.hadm_id, avg(los) as avg_los, sum(los) as tot_los
from TRANSFERS a
where a.los is not null
group by a.hadm_id"""
df2 = pysqldf(sql2)
df2

,subject_id,hadm_id,avg_los,tot_los
0,10056,100375,54.765000,109.53
1,42430,100969,44.685000,89.37
2,41914,101361,135.846667,407.54
3,42135,102203,41.244286,288.71
4,44228,103379,76.906667,230.72
...,...,...,...,...
124,44083,198330,60.280000,241.12
125,40177,198480,26.460000,132.30
126,10076,198503,56.407500,225.63
127,10017,199207,38.676000,193.38


In [114]:
sql3 = """
select df1.subject_id, df1.hadm_id, df1.num_transfers, df1.curr_careunit, df2.avg_los, df2.tot_los
from df1
left join df2 on df1.hadm_id = df2.hadm_id 
"""
df3 = pysqldf(sql3)
df3

,subject_id,hadm_id,num_transfers,curr_careunit,avg_los,tot_los
0,10056,100375,1,MICU,54.765000,109.53
1,42430,100969,1,SICU,44.685000,89.37
2,41914,101361,2,MICU,135.846667,407.54
3,42135,102203,6,MICU,41.244286,288.71
4,44228,103379,2,SICU,76.906667,230.72
...,...,...,...,...,...,...
124,44083,198330,3,CCU,60.280000,241.12
125,40177,198480,4,MICU,26.460000,132.30
126,10076,198503,3,MICU,56.407500,225.63
127,10017,199207,4,CCU,38.676000,193.38
